In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, MinMaxScaler, MaxAbsScaler
%matplotlib inline

In [2]:
import numpy as np
print(np.__version__)


2.0.0


In [3]:
df = pd.read_csv("/Users/abdullah/Documents/Personal Folder/GitHub/CSC417-MachineLearning/Datasets/Ames_Housing_Sales.csv")
# To display the top 5 rows
df.head(5)

,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2,...,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
0,856.0,854.0,0.0,NaN,3,1Fam,TA,No,706.0,0.0,...,0.0,Pave,8,856.0,AllPub,0.0,2003,2003,2008,208500.0
1,1262.0,0.0,0.0,NaN,3,1Fam,TA,Gd,978.0,0.0,...,0.0,Pave,6,1262.0,AllPub,298.0,1976,1976,2007,181500.0
2,920.0,866.0,0.0,NaN,3,1Fam,TA,Mn,486.0,0.0,...,0.0,Pave,6,920.0,AllPub,0.0,2001,2002,2008,223500.0
3,961.0,756.0,0.0,NaN,3,1Fam,Gd,No,216.0,0.0,...,0.0,Pave,7,756.0,AllPub,0.0,1915,1970,2006,140000.0
4,1145.0,1053.0,0.0,NaN,4,1Fam,TA,Av,655.0,0.0,...,0.0,Pave,9,1145.0,AllPub,192.0,2000,2000,2008,250000.0


In [4]:
print(df.shape)

(1379, 80)


In [5]:
df.dtypes.value_counts()

object     43
float64    21
int64      16
Name: count, dtype: int64

In [6]:
# Select the object (string) columns
mask = df.dtypes == object
categorical_cols = df.columns[mask]

In [7]:
# Determine how many extra columns would be created
num_ohe_cols = (df[categorical_cols]
                .apply(lambda x: x.nunique())
                .sort_values(ascending=False))

# No need to encode if there is only one value
small_num_ohe_cols = num_ohe_cols.loc[num_ohe_cols>1]

# Number of one-hot columns is one less than the number of categories
small_num_ohe_cols -= 1

# This is 215 columns, assuming the original ones are dropped. 
# This is quite a few extra columns!
small_num_ohe_cols.sum()

np.int64(204)

In [8]:
print(num_ohe_cols.index)

Index(['Neighborhood', 'Exterior2nd', 'Exterior1st', 'Condition1', 'SaleType',
       'Condition2', 'RoofMatl', 'HouseStyle', 'Functional', 'Heating',
       'BsmtFinType2', 'RoofStyle', 'GarageType', 'Foundation',
       'SaleCondition', 'LotConfig', 'MSZoning', 'HeatingQC', 'BldgType',
       'GarageQual', 'GarageCond', 'FireplaceQu', 'BsmtFinType1', 'Electrical',
       'BsmtExposure', 'MiscFeature', 'BsmtQual', 'LotShape', 'LandContour',
       'KitchenQual', 'ExterCond', 'ExterQual', 'Fence', 'LandSlope',
       'MasVnrType', 'PavedDrive', 'PoolQC', 'BsmtCond', 'GarageFinish',
       'Street', 'Alley', 'CentralAir', 'Utilities'],
      dtype='object')


In [9]:
data_ohe = df.copy()

le = LabelEncoder()
ohe = OneHotEncoder()

for col in num_ohe_cols.index:
    data = le.fit_transform(data_ohe[col]).astype(int) 

    data_ohe = data_ohe.drop(col, axis=1)

    new_data = ohe.fit_transform(data.reshape(-1,1))

    n_cols = new_data.shape[1]
    col_names = ['_'.join([col, str(x)]) for x in range(n_cols)]

    new_df = pd.DataFrame(new_data.toarray(), index=data_ohe.index, columns=col_names)

    # Append the new data to the dataframe
    data_ohe = pd.concat([data_ohe, new_df], axis=1)


In [10]:
# Column difference is as calculated above
data_ohe.shape[1] - df.shape[1]

215

In [11]:
print(df.shape[1])

# Remove the string columns from the dataframe
df = df.drop(num_ohe_cols.index, axis=1)

print(df.shape[1])

80
37


In [12]:
df.dtypes.value_counts()

float64    21
int64      16
Name: count, dtype: int64

In [13]:
y_col = 'SalePrice'

# Split the data that is not one-hot encoded
feature_cols = [x for x in df.columns if x != y_col]
X_data = df[feature_cols]
y_data = df[y_col]

X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, 
                                                    test_size=0.3, random_state=42)
# Split the data that is one-hot encoded
feature_cols = [x for x in data_ohe.columns if x != y_col]
X_data_ohe = data_ohe[feature_cols]
y_data_ohe = data_ohe[y_col]

X_train_ohe, X_test_ohe, y_train_ohe, y_test_ohe = train_test_split(X_data_ohe, y_data_ohe, 
                                                    test_size=0.3, random_state=42)

In [14]:
# Compare the indices to ensure they are identical
(X_train_ohe.index == X_train.index).all()

np.True_

In [14]:
print(X_train_ohe.index, "\n\n", X_train.index)

Index([ 461,  976, 1128,  904,  506,  894,  720,  333, 1336, 1063,
       ...
        330, 1238,  466,  121, 1044, 1095, 1130, 1294,  860, 1126],
      dtype='int64', length=965) 

 Index([ 461,  976, 1128,  904,  506,  894,  720,  333, 1336, 1063,
       ...
        330, 1238,  466,  121, 1044, 1095, 1130, 1294,  860, 1126],
      dtype='int64', length=965)


In [15]:
X_train.dtypes.value_counts()

float64    20
int64      16
Name: count, dtype: int64

In [16]:
LR = LinearRegression()

# Storage for error values
error_df = list()

# Data that have not been one-hot encoded
LR = LR.fit(X_train, y_train)
y_train_pred = LR.predict(X_train)
y_test_pred = LR.predict(X_test)

error_df.append(pd.Series({'train': mean_squared_error(y_train, y_train_pred),
                           'test' : mean_squared_error(y_test,  y_test_pred)},
                           name='no enc'))

# Data that have been one-hot encoded
LR = LR.fit(X_train_ohe, y_train_ohe)
y_train_ohe_pred = LR.predict(X_train_ohe)
y_test_ohe_pred = LR.predict(X_test_ohe)

error_df.append(pd.Series({'train': mean_squared_error(y_train_ohe, y_train_ohe_pred),
                           'test' : mean_squared_error(y_test_ohe,  y_test_ohe_pred)},
                          name='one-hot enc'))

# Assemble the results
error_df = pd.concat(error_df, axis=1)
error_df

,no enc,one-hot enc
train,1.131507e+09,3.177267e+08
test,1.372182e+09,8.065328e+09


In [8]:
num_columns = df.shape[1]
print(f"Number of columns: {num_columns}")

Number of columns: 80


In [9]:
column_names = df.columns.tolist()
print(f"Column names: {column_names}")

Column names: ['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr', 'BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtQual', 'BsmtUnfSF', 'CentralAir', 'Condition1', 'Condition2', 'Electrical', 'EnclosedPorch', 'ExterCond', 'ExterQual', 'Exterior1st', 'Exterior2nd', 'Fence', 'FireplaceQu', 'Fireplaces', 'Foundation', 'FullBath', 'Functional', 'GarageArea', 'GarageCars', 'GarageCond', 'GarageFinish', 'GarageQual', 'GarageType', 'GarageYrBlt', 'GrLivArea', 'HalfBath', 'Heating', 'HeatingQC', 'HouseStyle', 'KitchenAbvGr', 'KitchenQual', 'LandContour', 'LandSlope', 'LotArea', 'LotConfig', 'LotFrontage', 'LotShape', 'LowQualFinSF', 'MSSubClass', 'MSZoning', 'MasVnrArea', 'MasVnrType', 'MiscFeature', 'MiscVal', 'MoSold', 'Neighborhood', 'OpenPorchSF', 'OverallCond', 'OverallQual', 'PavedDrive', 'PoolArea', 'PoolQC', 'RoofMatl', 'RoofStyle', 'SaleCondition', 'SaleType', 'ScreenPorch', 'Street', 'To

In [13]:
for i in range(0, len(column_names), 5):
    print(column_names[i:i+5])
    display(df[column_names[i:i+5]].head())

['1stFlrSF', '2ndFlrSF', '3SsnPorch', 'Alley', 'BedroomAbvGr']


,1stFlrSF,2ndFlrSF,3SsnPorch,Alley,BedroomAbvGr
0,856.0,854.0,0.0,NaN,3
1,1262.0,0.0,0.0,NaN,3
2,920.0,866.0,0.0,NaN,3
3,961.0,756.0,0.0,NaN,3
4,1145.0,1053.0,0.0,NaN,4


['BldgType', 'BsmtCond', 'BsmtExposure', 'BsmtFinSF1', 'BsmtFinSF2']


,BldgType,BsmtCond,BsmtExposure,BsmtFinSF1,BsmtFinSF2
0,1Fam,TA,No,706.0,0.0
1,1Fam,TA,Gd,978.0,0.0
2,1Fam,TA,Mn,486.0,0.0
3,1Fam,Gd,No,216.0,0.0
4,1Fam,TA,Av,655.0,0.0


['BsmtFinType1', 'BsmtFinType2', 'BsmtFullBath', 'BsmtHalfBath', 'BsmtQual']


...

,PoolQC,RoofMatl,RoofStyle,SaleCondition,SaleType
0,NaN,CompShg,Gable,Normal,WD
1,NaN,CompShg,Gable,Normal,WD
2,NaN,CompShg,Gable,Normal,WD
3,NaN,CompShg,Gable,Abnorml,WD
4,NaN,CompShg,Gable,Normal,WD


['ScreenPorch', 'Street', 'TotRmsAbvGrd', 'TotalBsmtSF', 'Utilities']


,ScreenPorch,Street,TotRmsAbvGrd,TotalBsmtSF,Utilities
0,0.0,Pave,8,856.0,AllPub
1,0.0,Pave,6,1262.0,AllPub
2,0.0,Pave,6,920.0,AllPub
3,0.0,Pave,7,756.0,AllPub
4,0.0,Pave,9,1145.0,AllPub


['WoodDeckSF', 'YearBuilt', 'YearRemodAdd', 'YrSold', 'SalePrice']


,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
0,0.0,2003,2003,2008,208500.0
1,298.0,1976,1976,2007,181500.0
2,0.0,2001,2002,2008,223500.0
3,0.0,1915,1970,2006,140000.0
4,192.0,2000,2000,2008,250000.0


In [14]:

df.describe()

,1stFlrSF,2ndFlrSF,3SsnPorch,BedroomAbvGr,BsmtFinSF1,BsmtFinSF2,BsmtFullBath,BsmtHalfBath,BsmtUnfSF,EnclosedPorch,...,OverallQual,PoolArea,ScreenPorch,TotRmsAbvGrd,TotalBsmtSF,WoodDeckSF,YearBuilt,YearRemodAdd,YrSold,SalePrice
count,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,...,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.000000,1379.00000
mean,1177.129804,353.424946,3.609862,2.865120,455.578680,48.102248,0.430747,0.058738,570.765047,21.039159,...,6.187092,2.920957,15.945613,6.552574,1074.445975,97.456853,1972.958666,1985.435098,2007.812183,185479.51124
std,387.014961,439.553171,30.154682,0.783961,459.691379,164.324665,0.514052,0.238285,443.677845,60.535107,...,1.345780,41.335545,57.249593,1.589821,436.371874,126.699192,29.379883,20.444852,1.330221,79023.89060
min,438.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,2.000000,0.000000,0.000000,3.000000,0.000000,0.000000,1880.000000,1950.000000,2006.000000,35311.00000
25%,894.000000,0.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,228.000000,0.000000,...,5.000000,0.000000,0.000000,5.000000,810.000000,0.000000,1955.000000,1968.000000,2007.000000,134000.00000
50%,1098.000000,0.000000,0.000000,3.000000,400.000000,0.000000,0.000000,0.000000,476.000000,0.000000,...,6.000000,0.000000,0.000000,6.000000,1008.000000,0.000000,1976.000000,1994.000000,2008.000000,167500.00000
75%,1414.000000,738.500000,0.000000,3.000000,732.000000,0.000000,1.000000,0.000000,811.000000,0.000000,...,7.000000,0.000000,0.000000,7.000000,1314.000000,171.000000,2001.000000,2004.000000,2009.000000,217750.00000
max,4692.000000,2065.000000,508.000000,6.000000,5644.000000,1474.000000,2.000000,2.000000,2336.000000,552.000000,...,10.000000,738.000000,480.000000,12.000000,6110.000000,857.000000,2010.000000,2010.000000,2010.000000,755000.00000


In [15]:
duplicate_rows_df = df[df.duplicated()]
print("number of duplicate rows: ", duplicate_rows_df.shape)

number of duplicate rows:  (0, 80)


In [16]:

df.count()      # Used to count the number of rows

1stFlrSF        1379
2ndFlrSF        1379
3SsnPorch       1379
Alley             82
BedroomAbvGr    1379
                ... 
WoodDeckSF      1379
YearBuilt       1379
YearRemodAdd    1379
YrSold          1379
SalePrice       1379
Length: 80, dtype: int64

In [26]:
# Set options to display all rows
pd.set_option('display.max_rows', 100)  # Display all rows
pd.set_option('display.max_columns', 100)  # Display all columns

# Print the null values for each column
print(df.isnull().sum())

1stFlrSF            0
2ndFlrSF            0
3SsnPorch           0
Alley            1297
BedroomAbvGr        0
BldgType            0
BsmtCond          426
BsmtExposure      426
BsmtFinSF1          0
BsmtFinSF2          0
BsmtFinType1      426
BsmtFinType2      427
BsmtFullBath        0
BsmtHalfBath        0
BsmtQual          426
BsmtUnfSF           0
CentralAir          0
Condition1          0
Condition2          0
Electrical          0
EnclosedPorch       0
ExterCond           0
ExterQual           0
Exterior1st         0
Exterior2nd         0
Fence            1114
FireplaceQu       618
Fireplaces          0
Foundation          0
FullBath            0
Functional          0
GarageArea          0
GarageCars          0
GarageCond          0
GarageFinish        0
GarageQual          0
GarageType          0
GarageYrBlt         0
GrLivArea           0
HalfBath            0
Heating             0
HeatingQC           0
HouseStyle          0
KitchenAbvGr        0
KitchenQual         0
LandContou